In [ ]:
# --- Load required libraries ---
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import richdem as rd
from rasterio.plot import show
from scipy.ndimage import gaussian_filter

In [ ]:
# --- Load DEM ---
dem_path = 'your_dem.tif'  # Replace with your DEM file path
dem_raster = rasterio.open(dem_path)
dem = dem_raster.read(1)
profile = dem_raster.profile

# Mask no-data
dem = np.where(dem == profile['nodata'], np.nan, dem)

plt.figure(figsize=(10, 6))
plt.title("Digital Elevation Model (DEM)")
plt.imshow(dem, cmap='terrain')
plt.colorbar(label='Elevation (m)')
plt.axis('off')
plt.show()

In [ ]:
# --- Calculate Slope ---
from richdem.common import RDArray
rdem = rd.rdarray(dem, no_data=np.nan)
rdem.projection = profile['crs'].to_string()
slope = rd.TerrainAttribute(rdem, attrib='slope_degrees')

plt.figure(figsize=(10, 6))
plt.title("Slope (degrees)")
plt.imshow(slope, cmap='magma')
plt.colorbar(label='Degrees')
plt.axis('off')
plt.show()

In [ ]:
# --- Calculate TWI (Topographic Wetness Index) ---
# TWI = ln(A / tan(β))
# A = specific catchment area, β = slope in radians

catchment_area = rd.FlowAccumulation(rdem, method='D8')
slope_rad = np.radians(slope)
twi = np.log((catchment_area + 1) / (np.tan(slope_rad) + 0.001))  # Avoid division by zero

plt.figure(figsize=(10, 6))
plt.title("Topographic Wetness Index (TWI)")
plt.imshow(twi, cmap='Blues')
plt.colorbar(label='TWI')
plt.axis('off')
plt.show()

In [ ]:
# --- Relative Elevation Model (REM) ---
# Simplified: subtract a smoothed DEM from the original DEM

smoothed_dem = gaussian_filter(dem, sigma=30)
rem = dem - smoothed_dem

plt.figure(figsize=(10, 6))
plt.title("Relative Elevation Model (REM)")
plt.imshow(rem, cmap='RdBu', vmin=-10, vmax=10)
plt.colorbar(label='Relative Elevation (m)')
plt.axis('off')
plt.show()

In [ ]:
# --- Combine Wetness Indicators ---
# Normalize and combine TWI, REM, and slope

twi_norm = (twi - np.nanmin(twi)) / (np.nanmax(twi) - np.nanmin(twi))
rem_norm = (rem - np.nanmin(rem)) / (np.nanmax(rem) - np.nanmin(rem))
slope_norm = (slope - np.nanmin(slope)) / (np.nanmax(slope) - np.nanmin(slope))

# Weighting: high TWI, low slope, low REM
wetness_risk = (twi_norm * 0.5) + ((1 - slope_norm) * 0.25) + ((1 - rem_norm) * 0.25)

plt.figure(figsize=(10, 6))
plt.title("Wetness Risk Map")
plt.imshow(wetness_risk, cmap='cividis')
plt.colorbar(label='Wetness Score')
plt.axis('off')
plt.show()

In [ ]:
# --- Load and Rasterize Wetlands Vector Layer ---
import geopandas as gpd
from rasterio.features import rasterize

wetlands_path = 'wetlands.shp'  # Replace with your shapefile path
wetlands = gpd.read_file(wetlands_path)
wetlands = wetlands.to_crs(profile['crs'])  # Match DEM CRS

wetlands_mask = rasterize(
    [(geom, 1) for geom in wetlands.geometry],
    out_shape=dem.shape,
    transform=profile['transform'],
    fill=0,
    dtype='uint8'
)

In [ ]:
# --- Combine Wetness Risk Layer with Known Wetlands ---
combined_risk = np.maximum(wetness_risk, wetlands_mask)

# Save the result
from rasterio import Affine

output_path = 'combined_wet_risk.tif'
profile.update(dtype='uint8', count=1, nodata=0)

with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write((combined_risk > 0.5).astype('uint8'), 1)  # Threshold can be tuned

# Visualize result
plt.figure(figsize=(10, 6))
plt.imshow(combined_risk, cmap='Purples')
plt.title('Combined Wetness Risk + Wetlands')
plt.axis('off')
plt.colorbar(label='Wet Indicator')
plt.show()